In [1]:
import UCTB
import numpy as np
import tensorflow as tf
from UCTB.model_unit import GCLSTMCell
import traceback

Using TensorFlow backend.


In [2]:
num_hidden_unit = 32
num_node = 288
num_flow = 1
laplacian_matrix = tf.placeholder(tf.float32, [num_node, num_node], name='laplace_matrix')
time_step = 6
closeness_laplacian = tf.placeholder(tf.float32, [None, num_node, num_node, time_step], name='closeness_laplacian')
target_tensor = tf.placeholder(tf.float32, [None, num_node, time_step, num_flow], name='target_tensor')

In [3]:
static = GCLSTMCell(units=num_hidden_unit, num_nodes=num_node,
                                                laplacian_matrix=closeness_laplacian[0,:,:,0],
                                                dynamic_flag=False,
                                                gcn_k=1, gcn_l=1)

In [6]:
static.call()

TypeError: call() missing 2 required positional arguments: 'inputs' and 'states'

In [4]:
multi_layer_cell = [static]
tensor = tf.keras.layers.RNN(cell=multi_layer_cell)(tf.reshape(target_tensor, [-1,time_step,num_flow]))

Instructions for updating:
Colocations handled automatically by placer.
*****************static graph in ST_RNN*****************
*****************static graph in ST_RNN*****************


In [5]:
sess = tf.Session()
variable_init = tf.global_variables_initializer()
sess.run(variable_init)

In [6]:
traceback.print_stack()
result = sess.run(tensor,feed_dict={target_tensor:np.zeros((4,num_node,time_step,num_flow)),closeness_laplacian:np.zeros((1,num_node,num_node,time_step),dtype=np.float32),laplacian_matrix: np.zeros((num_node,num_node),dtype=np.float32)})


  File "c:\users\81512\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\81512\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\81512\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\81512\appdata\local\programs\python\python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\81512\appdata\local\programs\python\python36\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "c:\users\81512\appdata\local\programs\python\python36\lib\site-packages\tornado\platform\asyncio.py", line 149, in start
    self.asyncio_loop.run_forever()
  File "c:\users\81512\appdata\local\programs\python\python36\lib\asyncio\base_events.py", line 422, in run_f

In [7]:
result.shape

(1152, 32)

In [115]:
result.reshape([-1,num_node,num_hidden_unit]).shape

(4, 288, 32)

In [113]:
result.shape

(1152, 32)

In [110]:
multi_layer_cell[0].get_initial_state().sh

ValueError: batch_size and dtype cannot be None while constructing initial state: batch_size=None, dtype=None

In [43]:
tensor.shape

TensorShape([Dimension(288), Dimension(32)])

In [75]:
dynamic_cell = GCLSTMCell(units=num_hidden_unit, num_nodes=num_node,
                                                laplacian_matrix=closeness_laplacian[0,:,:,0],
                                                dynamic_flag=True,
                                                dynamic_laplacian = closeness_laplacian,
                                                time_step = time_step,
                                                gcn_k=1, gcn_l=1)

In [76]:
multi_layer_cell = [dynamic_cell]
dynamic_cell = tf.keras.layers.RNN(multi_layer_cell)


In [77]:
target_tensor

<tf.Tensor 'target_tensor_8:0' shape=(?, 288, 6, 1) dtype=float32>

In [84]:
num_flow

1

In [85]:
dynamic_cell(tf.reshape(target_tensor, [-1, time_step,num_flow]))

ValueError: Input 0 is incompatible with layer rnn_15: expected shape=(None, None, 6), found shape=[None, 6, 1]

In [33]:
time_step

6

In [13]:
dynamic_cell(tf.reshape(target_tensor, [-1, time_step, num_flow]))

****************** current step is : ****************** 0
****************** current step is : ****************** 1


<tf.Tensor 'rnn_2_3/strided_slice_7:0' shape=(288, 32) dtype=float32>

In [ ]:
dynamic_cell

In [15]:
sess.run(dynamic_cell,feed_dict={target_tensor:np.zeros((1,num_node,time_step,num_flow)),closeness_laplacian:np.zeros((1,num_node,num_node,time_step),dtype=np.float32),laplacian_matrix: np.zeros((num_node,num_node),dtype=np.float32)})


TypeError: unhashable type: 'numpy.ndarray'

In [24]:
multi_layer_cell = [cell]

In [25]:
target_tensor = np.zeros((1,num_node,time_step,2),dtype=np.float32)

In [27]:
tf.keras.layers.RNN(multi_layer_cell)

In [26]:
tf.keras.layers.RNN(multi_layer_cell)(tf.reshape(target_tensor, [-1, time_step, num_flow]))

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "c:\users\81512\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\keras\backend.py", line 3443, in <genexpr>
    for ta, input_ in zip(input_ta, flatted_inputs))  File "c:\users\81512\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\util\tf_should_use.py", line 193, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs))
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "c:\users\81512\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\keras\backend.py", line 3443, in <ge

ValueError: Dimensions must be equal, but are 6 and 288 for 'rnn_3/MatMul_2' (op: 'MatMul') with input shapes: [288,6], [288,2].